# I. INTRODUCTION

# II. IMPORT LIBRARY

In [479]:
# Data manipulation and visualization
import pandas as pd
import numpy as np
import re

# Statistics
from scipy.stats import (
    iqr,
    pearsonr,
    spearmanr,
    kendalltau,
    normaltest
)

# Data splitting
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

# Imputer
from sklearn.impute import KNNImputer

# Preprocessing
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# Regression models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

# Metrics
from sklearn.metrics import r2_score, mean_absolute_error

# Saving and loading models
import pickle


# III. DATA LOADING

In [480]:
pd.set_option("display.max_columns", 500)
raw_data= pd.read_csv("Property_Scraping.csv")
data = raw_data.copy()
data

,url,title,description,price,address,city,land_size_m2,building_size_m2,bedroom,bathroom,garage,carport,property_type,certificate,voltage_watt,maid_bedroom,maid_bathroom,kitchen,dining_room,living_room,furniture,building_material,floor_material,floor_level,house_facing,concept_and_style,view,internet_access,road_width,year_built,year_renovated,water_source,corner_property,property_condition,ad_type,ad_id
0,/properti/tangerang/hos16543378/,"Rumah 2 Lantai Bagus Semi Furnished Sertifikat Hak Milik di Daan Mogot, Tangerang",Dijual Termurah Rumah Budi Indah Daan Mogot Siap Huni Rapi 3.65 X 15 Lt. 57 M² Lb. 114 M² 2 Lantai 3+1 Kamar Tidur 2 Kamar Mandi 2200 Watt Pam •Ada Bak Penampungan + Torent Dengan Kapasitas 1100L Garasi 1 Mobil Shm -Terletak Didalam Komplek -Full Renovasi -Area Sangat Strategis -Bebas Banjir -Row Jalan 2 Mobil -Asri Nyaman Dan Tenang -Security 24 Jam -One Gate System -Dekat Sekolah Dan Rumah Sakit Terdekat *Harga : 850Jt Nego (Termurah) ‼️ Property Consultant: Aling.Ng 0856 711 0xxx •Terima Titip Property Jual -Beli -Sewa : Rumah-Ruko-Apartemen-Soho- Gudang-Pabrik-Kavling-Office Space-Gedung-Hotel-Villa-Citra Garden - Pik1-Pluit-Muara Karang-Pik2-Golf Island-Bsd-Tangerang- Gading Serpong-Alam Sutera- Lippo Karawaci-Jabodetabek,8.500000e+08,"Daan Mogot, Tangerang",Tangerang,57,114.0,3.0,2.0,1.0,1.0,Rumah,SHM - Sertifikat Hak Milik,2200.0,1.0,0.0,0.0,0.0,0.0,Semi Furnished,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Row Jalan 2 Mobil,NaN,NaN,Pam,False,Full Renovasi,NaN,NaN
1,/properti/jakarta-selatan/hos18392523/,Rumah Asri Dan Eklusive Dikawasan Pemukiman Pondok Pinang Selangkah Ke Pondok Indah Dan Mrt Lebak Bulus Dan Tol Tb Simatupang,"Dijual Rumah Asri dan Nyaman di lingkungan yang Tenang ,di Kawasan Pemukiman kebayoran lama dekat ke Pondok Indah dan Lebakbulus . Bangunan Rumah sangat kokoh , Kekar dan terawat juga Halaman depan dan belakang tertata dengan baik . .Desain bangunan sangat Moderen dan Eklusive terdiri dari 2 lantai. Jalan depan Rumah 2 Mobil . Bangunan sekitarnya sangat tertata dengan baik dan teratur . Lokasi sangat strategis sekali karena : - Dekat ke Mall Pondok Indah , Gandaria City dan Cilandak Mall - Dekat ke RS Fatmawaty , RS Siloam , RS Pondok Indah dan RS Mayapada - Dekat ke Stasiun MRT Lebak Bulus dan Stasiun Kereta Kebayoran lama - Dekat ke Tol TB Simatupang dan Tol Lingkar Luar JABODETABEK - Dekat ke Kampus BINUS dan Kampus Prasetiya Mulya -Lokasi sangat strategis sekali sangat cocok buat tempat tinggal, Kos2an Eklusive dan Kantor karena dekat dari jl. Raya Ciputat Kebayoran lama. Rumah dijual cepat untuk info hub : Pesta LR RW Kebayoran Blok M .",4.950000e+09,"Kebayoran Lama, Jakarta Selatan",Jakarta Selatan,320,450.0,5.0,3.0,0.0,2.0,Rumah,SHM - Sertifikat Hak Milik,4400.0,0.0,0.0,0.0,0.0,0.0,Unfurnished,NaN,NaN,2.0,NaN,Moderen dan Eklusive,NaN,NaN,2 Mobil,NaN,NaN,NaN,NaN,terawat,Dijual Cepat,NaN
2,/properti/depok/hos18570241/,Rumah Di Perum Lereng Indah Cinere Perbatasan Pondok Cabe Ciputat,"Dijual Cepat Rumah Di Perum Lereng Indah, Pondok Cabe, Cinere Posisi rumah terletak di perbatasan Pondok Cabe , Pamulang , Ciputat ( Tangerang Selatan) dan Cinere ( Depok ) LT : 78m2 (6*13) LB : +/- 36m2 KT : 2 KM : 1 Bangunan 1 lantai Listrik : 1200 watt Hadap : Timur Carport : 1mobil Row : 1mobil Air tanah SHM Harga Jual : 590 jt (Nego)",5.900000e+08,"Perum Lereng Indah, Pondok Cabe, Cinere",Depok,78,36.0,2.0,1.0,0.0,1.0,Rumah,SHM - Sertifikat Hak Milik,1200.0,0.0,0.0,0.0,0.0,0.0,Unfurnished,NaN,NaN,1.0,East,NaN,NaN,NaN,1mobil,NaN,NaN,Air tanah,False,NaN,Dijual Cepat,NaN
3,/properti/jakarta-utara/hos16808954/,DIJUAL TERMURAH RUMAH PIK1 Layar Permai 2 Lantai Shm Sudah Renovasi Rumah Unfurnished Bagus SHM - Sertifikat Hak Milik di Pantai Indah Kapuk,DIJUAL TERMURAH RUMAH PIK1 Layar Permai 4x12 2 Lantai 2+1 Kamar tidur 2 Kamar mandi SHM •Sudah Renovasi | Rapi Terawat •Berada didalam Komplek Asri Bersih Nyaman dan Aman •Lingkungan sangat Asri Sejuk banyak Pohon Rindang •Row Jalan 2 Mobil •Security 24 jam | 1 Gat

# IV. EDA

## Cek Data Info

In [481]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1398 entries, 0 to 1397
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   url                 1398 non-null   object 
 1   title               1396 non-null   object 
 2   description         1396 non-null   object 
 3   price               1396 non-null   float64
 4   address             1299 non-null   object 
 5   city                1396 non-null   object 
 6   land_size_m2        1398 non-null   int64  
 7   building_size_m2    1398 non-null   float64
 8   bedroom             1224 non-null   float64
 9   bathroom            1394 non-null   float64
 10  garage              1330 non-null   float64
 11  carport             1392 non-null   float64
 12  property_type       1046 non-null   object 
 13  certificate         1395 non-null   object 
 14  voltage_watt        909 non-null    float64
 15  maid_bedroom        1393 non-null   float64
 16  maid_b

## Cek Duplicate

In [482]:
data.duplicated().sum()

0

In [483]:
data.url.duplicated().sum()

2

## Handling Duplicate

In [484]:
data = data.drop_duplicates()

print(data.duplicated().sum())

0


In [485]:
data = data.drop_duplicates(subset="url")
data.url.duplicated().sum()

0

## Cek Unique dan Cardinality

In [486]:
def unique_check_all(df):
    # Tampilan dengan max column
    pd.set_option('display.max_colwidth', None)

    # output bernama listItem
    listItem = []

    # Looping column di dalam dataframe
    for col in df.columns:
        # Menambahkan 'nama kolom', 'jumlah nilai unique', 'nilai unique' ke listItem
        listItem.append([col, df[col].nunique(), df[col].unique()])

    # Hasil dalam dataframe
    summary_cardinality= pd.DataFrame(columns=['nama kolom', 'jumlah nilai unique', 'nilai unique'], data=listItem)

    return summary_cardinality

In [487]:
unique_check_all(data)

nama kolom  jumlah nilai unique  \
0                  url                 1396   
1                title                 1345   
2          description                 1339   
3                price                  429   
4              address                  534   
5                 city                   42   
6         land_size_m2                  420   
7     building_size_m2                  342   
8              bedroom                   29   
9             bathroom                   26   
10              garage                   13   
11             carport                   13   
12       property_type                    3   
13         certificate                   25   
14        voltage_watt                   61   
15        maid_bedroom                    6   
16       maid_bathroom                    5   
17             kitchen                    4   
18         dining_room                    3   
19         living_room                    4   
20           furniture                    3   
21   building_material                   47   
22      floor_material                   39   
23         floor_level                   29   
24        house_facing                    8   
25   concept_and_style                   69   
26                view                   64   
27     internet_access                   24   
28          road_width                  145   
29          year_built                   24   
30      year_renovated                    5   
31        water_source                  120   
32     corner_property                    2   
33  property_condition                  126   
34             ad_type                   17   
35               ad_id                  567   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## Handling Cardinality

### 1. Price

In [488]:
# Rename the column and convert the values to millions
data = data.rename(columns={"price": "price_mio"})
data["price_mio"] = data["price_mio"] / 1_000_000

### 2. City

In [489]:
data.loc[:, 'city'] = data['city'].apply(lambda x: x.split(', ')[1] if isinstance(x, str) and ', ' in x else x)

# List kata kunci untuk kota Jabodetabek (menggunakan kata kunci seperti 'Jakarta' dan 'Bogor')
jabodetabek_keywords = ['Jakarta', 'Bogor', 'Depok', 'Tangerang', 'Bekasi']

# Filter untuk hanya menyertakan data dengan kota yang mengandung kata kunci Jabodetabek
data = data[data['city'].str.contains('|'.join(jabodetabek_keywords), case=False, na=False)]

In [490]:
# Keywords for classification
keywords = {
    r'jakarta': 'Jakarta',
    r'bogor': 'Bogor',
    r'depok': 'Depok',
    r'tangerang': 'Tangerang',
    r'bekasi': 'Bekasi'
}

# Step 1: Classification based on 'address' column
for pattern, city_name in keywords.items():
    data.loc[
        data['city'].isna() & data['address'].str.contains(pattern, case=False, na=False),
        'city'
    ] = city_name

# Step 2: Classification based on 'title' column
for pattern, city_name in keywords.items():
    data.loc[
        data['city'].isna() & data['title'].str.contains(pattern, case=False, na=False),
        'city'
    ] = city_name

# Step 3: Standardize the city column using regex
for pattern, city_name in keywords.items():
    data['city'] = data['city'].str.replace(rf'.*{pattern}.*', city_name, case=False, regex=True)

# Step 4: Remove any additional spaces and ensure consistency
data['city'] = data['city'].str.strip()

# Verify the output
print(data['city'].value_counts())

city
Jakarta      823
Tangerang    224
Bogor        124
Bekasi        94
Depok         74
Name: count, dtype: int64


### 3. Property Type

In [491]:
# Daftar kata kunci
keywords = {
    r'rumah|house|mansion': 'Rumah',
    r'apartment|apartmen|apartement|apartemen|kos|kost': 'Hunian Sewa',
    r'pabrik|kantor|office|ruko|ruang usaha|kios|kiosk|gudang': 'Ruang Usaha',
    r'tanah|lahan|kavling|gedung': 'Tanah dan Properti Lain'
}

# Normalisasi awal
data['property_type'] = data['property_type'].str.lower().str.strip()

# Klasifikasi berdasarkan 'property_type'
for pattern, prop_type in keywords.items():
    data.loc[
        data['property_type'].str.contains(pattern, case=False, na=False),
        'property_type'
    ] = prop_type

# Klasifikasi berdasarkan 'title'
for pattern, prop_type in keywords.items():
    data.loc[
        data['property_type'].isna() & data['title'].str.contains(pattern, case=False, na=False),
        'property_type'
    ] = prop_type

# Klasifikasi berdasarkan 'description'
for pattern, prop_type in keywords.items():
    data.loc[
        data['property_type'].isna() & data['description'].str.contains(pattern, case=False, na=False),
        'property_type'
    ] = prop_type

# Klasifikasi tipe rumah berdasarkan ukuran bangunan
def determine_house_type(row):
    if row['property_type'] == 'Rumah':
        size = row.get('building_size_m2', None)
        if size is None:
            return 'Rumah Tipe Tidak Diketahui'
        elif size <= 21:
            return 'Rumah Tipe 21'
        elif 21 < size <= 36:
            return 'Rumah Tipe 36'
        elif 36 < size <= 45:
            return 'Rumah Tipe 45'
        elif 45 < size <= 54:
            return 'Rumah Tipe 54'
        elif 54 < size <= 60:
            return 'Rumah Tipe 60'
        elif 60 < size <= 70:
            return 'Rumah Tipe 70'
        elif 70 < size <= 120:
            return 'Rumah Tipe 120'
        else:
            return 'Rumah Tipe >120'
    return row['property_type']  # Return original property_type for non-Rumah rows

# Update property_type with house_type where applicable
data['property_type'] = data.apply(determine_house_type, axis=1)

# Output hasil
data.property_type.value_counts()

property_type
Rumah Tipe >120            706
Rumah Tipe 120             233
Ruang Usaha                100
Tanah dan Properti Lain     90
Hunian Sewa                 79
Rumah Tipe 70               34
Rumah Tipe 60               26
Rumah Tipe 36               24
Rumah Tipe 54               24
Rumah Tipe 45               17
Rumah Tipe 21                6
Name: count, dtype: int64

In [492]:
data = data[(data['property_type'] != 'Tanah dan Properti Lain') &
                 (data['property_type'] != 'Hunian Sewa') &
                 (data['property_type'] != 'Ruang Usaha')]

data.property_type.unique()

array(['Rumah Tipe 120', 'Rumah Tipe >120', 'Rumah Tipe 36',
       'Rumah Tipe 70', 'Rumah Tipe 54', 'Rumah Tipe 45', 'Rumah Tipe 60',
       'Rumah Tipe 21'], dtype=object)

### 4. Certificate

In [493]:
# Define a function to categorize values
def certificate(value):
    if pd.isna(value):  # Handle NaN values
        return np.nan
    elif 'SHM' in value:  # If the value contains 'SHM'
        return 'SHM'
    elif 'HGB' in value:  # If the value contains 'HGB'
        return 'HGB'
    else:  # If the value doesn't contain 'SHM' or 'HGB'
        return 'Other'

# Apply the function to the 'certificate' column directly
data.loc[:,'certificate'] = data['certificate'].apply(certificate)

data.certificate.value_counts()

certificate
SHM      885
Other    133
HGB       49
Name: count, dtype: int64

### 5. Property Condition

In [494]:
# Fungsi untuk mengelompokkan berdasarkan kesamaan kata
def categorize(value):
    if pd.isnull(value):  # Tetap NaN jika value adalah NaN
        return pd.NA
    
    value_lower = str(value).lower()
    
    # Renovated: Properti yang telah direnovasi
    renovated_keywords = ['renov', 'full renov', 'renovasi', 'renoved', 'renovasi baru', 'baru renovasi', 'finished', 'selesai renovasi', 'proses finishing']
    if any(keyword in value_lower for keyword in renovated_keywords):
        return 'Renovated'
    
    # New: Properti yang baru atau baru dibangun
    new_keywords = ['new', 'brand new', 'baru', 'unit baru', 'first time', 'primery', 'full baru', 'baru selesai', 'unit baru gress']
    if any(keyword in value_lower for keyword in new_keywords):
        return 'New'
    
    # Need Renovation: Properti yang membutuhkan renovasi atau perawatan
    need_renovation_keywords = ['butuh renovasi', 'harus renovasi', 'setengah jadi', 'perlu renovasi', 'perlu perawatan', 'lama', 'tua']
    if any(keyword in value_lower for keyword in need_renovation_keywords):
        return 'Need Renovation'
    
    # Well Maintained: Properti yang terawat dan siap huni
    well_maintained_keywords = ['terawat', 'siap huni', 'bersih', 'rapi', 'kokoh', 'bagus', 'layak huni', 'ready to move', 'well maintained', 'layak', 'baik', 'well']
    if any(keyword in value_lower for keyword in well_maintained_keywords):
        return 'Well Maintained'
    
    # Jika tidak cocok dengan kategori apapun, beri nilai NaN (opsional)
    return pd.NA

# Terapkan fungsi kategori
data.loc[:,'property_condition'] = data['property_condition'].apply(lambda x: categorize(x))

data.property_condition.value_counts()

property_condition
Well Maintained    138
New                103
Renovated           25
Need Renovation     17
Name: count, dtype: int64

### 6. Water Source

In [495]:
keywords_water = {
    'PAM/PDAM': r'\b(?:pam|pdam|air pam|air pdam|pln pam|aetra|water treatment|palyja)\b',
    'Sumber Air': r'\b(?:sumur|jet pump|jetpump|sumur bor|air sumur|bor|tanah|air tanah|filter|osmosis|reverse osmosis|sistem filter|pompa|submersible pump|water pump|mata air|air alami|wtp|jetpam|air jet pum|air ready|langsung dari sumbernya|air bagus|sumber air)\b',
    'Gabungan': r'\b(?:pdam\s?\+?\s?sumur|pam\s?\+?\s?tanah|air jetpump)\b'
}

data['water_source'] = data['water_source'].str.lower().str.strip()

# Check and classify rows where 'water_source' is not missing
for category, pattern in keywords_water.items():
    data.loc[data['water_source'].str.contains(pattern, case=False, na=False), 'water_source'] = category

# Step 2: Handle missing 'water_source' values by checking the 'description' for matching keywords
for category, pattern in keywords_water.items():
    data.loc[data['water_source'].isna() & data['description'].str.contains(pattern, case=False, na=False), 'water_source'] = category

# Periksa distribusi kategori
data.water_source.value_counts()


water_source
Sumber Air    553
PAM/PDAM      238
Name: count, dtype: int64

### 7. Road Width

In [496]:
def convert_to_meter(value):
    if pd.isnull(value):  # Jika nilai NaN
        return np.nan
    
    value_lower = str(value).lower()
    
    # Ukuran dalam meter eksplisit
    if 'meter' in value_lower or 'mtr' in value_lower:
        match = re.search(r'(\d+\.?\d*)\s?(meter|mtr)', value_lower)
        if match:
            return float(match.group(1))
    
    # Kondisi 1 mobil
    if any(keyword in value_lower for keyword in ['1 mobil', '1mobil', '1 mbl', '1 arah mobil']):
        return 2.5
    
    # Kondisi 2 mobil
    if any(keyword in value_lower for keyword in ['2 mobil', '2 mobil lega', '2-3 mobil', '2 mobil pas', '2 mbl', 'akses jalan 2 mobil', '2 mobil 2 arah', 'row jalan 2 mobil', '2.5 mobil', '2mob', '2row']):
        return 5
    
    # Kondisi 3 mobil
    if any(keyword in value_lower for keyword in ['3 mobil', '3 row', '3 mbl', 'jalan 3 mobil', 'row jalan 3 mobil', '3 mobil lebih', '3mob']):
        return 7.5
    
    # Kondisi 4 mobil
    if any(keyword in value_lower for keyword in ['4 mobil', '4 mbl']):
        return 10
    
    # Kondisi lebih dari 4 mobil
    if any(keyword in value_lower for keyword in ['5 mobil', '6 mobil', '7 mobil', '8 mobil', 'lebih dari 4 mobil']):
        return 12
    
    # Kondisi lebar jalan besar atau akses jalan lebar
    if any(keyword in value_lower for keyword in ['lebar', 'besar', 'akses jalan', 'jalan besar']):
        return 5
    
    # Kondisi "super lebar"
    if 'super lebar' in value_lower:
        return 15
    
    # Jika tidak cocok dengan kategori apapun, beri nilai NaN (opsional)
    return np.nan

# Terapkan konversi ke meter
data.loc[:,'road_width'] = data['road_width'].apply(convert_to_meter)
# Convert 'voltage_watt' to float, with error handling for non-numeric values
data.road_width = pd.to_numeric(data['road_width'], errors='coerce')

data["road_width"].unique()

array([ 5. ,  2.5,  nan,  7.5,  8. , 45. ,  6. ,  7. , 10. ,  9. , 12. ])

## Cek Missing Value

In [497]:
data.isnull().sum()

url                      0
title                    0
description              0
price_mio                0
address                 65
city                     0
land_size_m2             0
building_size_m2         0
bedroom                 23
bathroom                 2
garage                  66
carport                  4
property_type            0
certificate              3
voltage_watt           291
maid_bedroom             3
maid_bathroom           16
kitchen                144
dining_room            199
living_room            192
furniture              210
building_material     1004
floor_material        1012
floor_level             31
house_facing           721
concept_and_style      935
view                  1023
internet_access       1027
road_width             847
year_built            1027
year_renovated        1061
water_source           279
corner_property        715
property_condition     787
ad_type                586
ad_id                  785
dtype: int64

## Mendefinisikan Kolom kategorikal dan Numerikal

In [498]:
# Select numeric columns
num_cols = ['land_size_m2',
            'building_size_m2',
            'road_width',
            'maid_bedroom',
            'maid_bathroom',
            'kitchen',
            'floor_level',
            'bedroom',
            'bathroom',
            'garage',
            'carport',
            'voltage_watt']

cat_cols = ['city',
            'property_type',
            'certificate',
            'furniture',
            'house_facing',
            'water_source',
            'property_condition'
            ]

In [499]:
df = data[['land_size_m2',
          'building_size_m2',
          'road_width',
          'city',
          'property_type',
          'certificate',
          'furniture',
          'house_facing',
          'water_source',
          'property_condition',
          'bedroom',
          'bathroom',
          'garage',
          'carport',
          'voltage_watt',
          'maid_bedroom',
          'maid_bathroom',
          'kitchen',
          'floor_level',
          'price_mio']]

# V. FEATURE ENGINEERING

## Split antara X (Features) dan y (Target)

In [500]:
# Split kolom features dan target
X = df.drop('price_mio', axis=1) # Features
y = df['price_mio']               # Target

In [501]:
# Split dataset into training+validation and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=999)

In [502]:
X_train

,land_size_m2,building_size_m2,road_width,city,property_type,certificate,furniture,house_facing,water_source,property_condition,bedroom,bathroom,garage,carport,voltage_watt,maid_bedroom,maid_bathroom,kitchen,floor_level
562,136,220.0,NaN,Bogor,Rumah Tipe >120,SHM,Semi Furnished,South,PAM/PDAM,<NA>,4.0,2.0,0.0,1.0,2200.0,0.0,0.0,0.0,2.0
1326,320,250.0,7.5,Jakarta,Rumah Tipe >120,SHM,Semi Furnished,East,NaN,<NA>,3.0,3.0,2.0,3.0,4400.0,1.0,1.0,2.0,2.0
913,126,146.0,NaN,Jakarta,Rumah Tipe >120,Other,Unfurnished,South,Sumber Air,New,4.0,4.0,0.0,0.0,3500.0,1.0,1.0,0.0,2.5
823,165,132.0,NaN,Tangerang,Rumah Tipe >120,SHM,Unfurnished,NaN,Sumber Air,<NA>,3.0,2.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0
722,336,400.0,5.0,Jakarta,Rumah Tipe >120,SHM,Semi Furnished,NaN,Sumber Air,Well Maintained,4.0,4.0,0.0,4.0,NaN,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,300,310.0,NaN,Tangerang,Rumah Tipe >120,SHM,Semi Furnished,NaN,NaN,<NA>,4.0,4.0,0.0,2.0,3500.0,1.0,1.0,0.0,0.0
281,115,200.0,NaN,Tangerang,Rumah Tipe >120,Other,NaN,NaN,Sumber Air,<NA>,5.0,4.0,0.0,0.0,NaN,0.0,0.0,2.0,0.0
601,213,378.0,NaN,Jakarta,Rumah Tipe >120,SHM,Semi Furnished,NaN,Sumber Air,<NA>,4.0,4.0,NaN,2.0,16000.0,2.0,1.0,NaN,3.0
1125,126,180.0,5.0,Bogor,Rumah Tipe >120,SHM,NaN,NaN,PAM/PDAM,<NA>,2.0,2.0,0.0,2.0,2200.0,0.0,0.0,2.0,2.0


## Handling Missing Value

In [503]:
def impute_with_knn(X_train, X_test, num_cols, cat_cols):
    # Create deep copies to avoid modifying original dataframes
    X_train_imputed = X_train.copy(deep=True)
    X_test_imputed = X_test.copy(deep=True)
    
    # Dictionary to store LabelEncoders for each categorical column
    label_encoders = {}
    
    # Encode categorical columns
    def encode_categorical(df, cat_cols, label_encoders):
        df_encoded = df.copy(deep=True)
        for col in cat_cols:
            if col not in label_encoders:
                # Combine unique categories from train and test to fit encoder
                combined_categories = pd.concat([X_train[col], X_test[col]]).unique()
                le = LabelEncoder()
                le.fit(combined_categories[~pd.isna(combined_categories)].astype(str))  # Ignore NaN
                label_encoders[col] = le
            
            # Encode the column (leave NaN as is for imputing later)
            le = label_encoders[col]
            df_encoded[col] = df_encoded[col].apply(
                lambda x: le.transform([str(x)])[0] if pd.notna(x) else np.nan
            )
        return df_encoded

    # Encode train and test datasets
    X_train_encoded = encode_categorical(X_train_imputed, cat_cols, label_encoders)
    X_test_encoded = encode_categorical(X_test_imputed, cat_cols, label_encoders)
    
    # Combine numeric and encoded categorical columns for imputing
    cols_to_impute = num_cols + cat_cols
    X_train_to_impute = X_train_encoded[cols_to_impute]
    X_test_to_impute = X_test_encoded[cols_to_impute]
    
    # Apply KNN Imputer
    knn_imputer = KNNImputer(n_neighbors=5)
    X_train_imputed_values = knn_imputer.fit_transform(X_train_to_impute)
    X_test_imputed_values = knn_imputer.transform(X_test_to_impute)
    
    # Update the imputed values back to dataframes
    X_train_imputed[cols_to_impute] = X_train_imputed_values
    X_test_imputed[cols_to_impute] = X_test_imputed_values
    
    # Decode categorical columns back to original values
    def decode_categorical(df, cat_cols, label_encoders):
        df_decoded = df.copy(deep=True)
        for col in cat_cols:
            le = label_encoders[col]
            df_decoded[col] = df_decoded[col].apply(
                lambda x: le.inverse_transform([int(x)])[0] if pd.notna(x) else np.nan
            )
        return df_decoded

    # Decode train and test datasets
    X_train_imputed = decode_categorical(X_train_imputed, cat_cols, label_encoders)
    X_test_imputed = decode_categorical(X_test_imputed, cat_cols, label_encoders)
    
    return X_train_imputed, X_test_imputed


In [504]:
X_train_imputed, X_test_imputed = impute_with_knn(X_train, X_test, num_cols, cat_cols)


In [506]:
X_train_imputed

,land_size_m2,building_size_m2,road_width,city,property_type,certificate,furniture,house_facing,water_source,property_condition,bedroom,bathroom,garage,carport,voltage_watt,maid_bedroom,maid_bathroom,kitchen,floor_level
562,136.0,220.0,4.5,Bogor,Rumah Tipe >120,SHM,Semi Furnished,South,PAM/PDAM,New,4.0,2.0,0.0,1.0,2200.0,0.0,0.0,0.0,2.0
1326,320.0,250.0,7.5,Jakarta,Rumah Tipe >120,SHM,Semi Furnished,East,PAM/PDAM,Renovated,3.0,3.0,2.0,3.0,4400.0,1.0,1.0,2.0,2.0
913,126.0,146.0,5.0,Jakarta,Rumah Tipe >120,Other,Unfurnished,South,Sumber Air,New,4.0,4.0,0.0,0.0,3500.0,1.0,1.0,0.0,2.5
823,165.0,132.0,5.0,Tangerang,Rumah Tipe >120,SHM,Unfurnished,Northeast,Sumber Air,New,3.0,2.0,0.0,0.0,2920.0,0.0,0.0,0.0,1.0
722,336.0,400.0,5.0,Jakarta,Rumah Tipe >120,SHM,Semi Furnished,Northwest,Sumber Air,Well Maintained,4.0,4.0,0.0,4.0,7180.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,300.0,310.0,5.5,Tangerang,Rumah Tipe >120,SHM,Semi Furnished,East,PAM/PDAM,Renovated,4.0,4.0,0.0,2.0,3500.0,1.0,1.0,0.0,0.0
281,115.0,200.0,5.0,Tangerang,Rumah Tipe >120,Other,Semi Furnished,Northeast,Sumber Air,New,5.0,4.0,0.0,0.0,4080.0,0.0,0.0,2.0,0.0
601,213.0,378.0,5.0,Jakarta,Rumah Tipe >120,SHM,Semi Furnished,North,Sumber Air,Renovated,4.0,4.0,0.4,2.0,16000.0,2.0,1.0,0.6,3.0
1125,126.0,180.0,5.0,Bogor,Rumah Tipe >120,SHM,Semi Furnished,Southeast,PAM/PDAM,New,2.0,2.0,0.0,2.0,2200.0,0.0,0.0,2.0,2.0


## Cek Outlier dan Skewness

In [507]:

def check_outlier(data, cols):
    '''
    Fungsi ini digunakan untuk mengecek nilai lower bound dan upper bound data serta persentase data outlier. 
    Fungsi ini hanya bisa digunakan untuk kolom numeric. Pastikan parameter cols hanya berisi kolom dengan tipe data numeric.
    Fungsi ini akan mengabaikan nilai missing value.
    '''
    
    skewness = [data[col].dropna().skew() for col in cols]  # Mengabaikan nilai NaN saat menghitung skewness

    listItem = []  # List untuk menyimpan setiap row hasil per kolom

    for id, col in enumerate(cols):
        # Mengabaikan nilai NaN pada kolom yang sedang dianalisis
        col_data = data[col].dropna()

        if skewness[id] > 1 or skewness[id] < -1:
            category = 'Highly Skewed'
            IQR = iqr(col_data)
            lower_boundary = col_data.quantile(0.25) - 3 * IQR
            upper_boundary = col_data.quantile(0.75) + 3 * IQR
        elif skewness[id] > 0.5 or skewness[id] < -0.5:
            category = 'Moderately Skewed'
            IQR = iqr(col_data)
            lower_boundary = col_data.quantile(0.25) - 1.5 * IQR
            upper_boundary = col_data.quantile(0.75) + 1.5 * IQR
        else:
            category = 'Normal'
            lower_boundary = col_data.mean() - 3 * col_data.std()
            upper_boundary = col_data.mean() + 3 * col_data.std()

        # Menghitung outlier pada data yang telah di-drop NaN-nya
        outlier = col_data[(col_data > upper_boundary) | (col_data < lower_boundary)]
        outlier_num = len(outlier)
        outlier_percent = outlier_num / len(col_data) * 100  # Menggunakan panjang kolom tanpa NaN

        # Append hasil analisis ke listItem sebagai satu row
        listItem.append([
            col,                      # Nama Kolom
            round(skewness[id], 2),    # Skewness
            category,                  # Kategori Distribusi
            round(lower_boundary, 2),  # Lower Boundary
            round(upper_boundary, 2),  # Upper Boundary
            outlier_num,               # Jumlah Outlier
                        round(outlier_percent, 2)  # Persentase Outlier
        ])

    # Membuat DataFrame dari listItem
    df_distribution = pd.DataFrame(listItem, columns=[
        'Column', 'Skewness', 'Distribution', 'LowerBoundary', 'UpperBoundary', 'Jumlah Outlier', 'Percent Outlier (%)'
    ])

    return df_distribution

In [508]:
X_train_distribution = check_outlier(X_train_imputed,num_cols)

print("Distribusi data value pada X_train:")
X_train_distribution

Distribusi data value pada X_train:


,Column,Skewness,Distribution,LowerBoundary,UpperBoundary,Jumlah Outlier,Percent Outlier (%)
0,land_size_m2,6.72,Highly Skewed,-348.0,688.0,34,3.53
1,building_size_m2,3.16,Highly Skewed,-500.0,900.0,22,2.28
2,road_width,7.18,Highly Skewed,3.5,7.0,97,10.07
3,maid_bedroom,1.55,Highly Skewed,-3.0,4.0,2,0.21
4,maid_bathroom,1.26,Highly Skewed,-3.0,4.0,0,0.00
5,kitchen,2.18,Highly Skewed,-0.6,0.8,162,16.82
6,floor_level,10.76,Highly Skewed,-6.0,8.0,7,0.73
7,bedroom,10.77,Highly Skewed,0.0,7.0,43,4.47
8,bathroom,11.87,Highly Skewed,-4.0,10.0,17,1.77
9,garage,4.24,Highly Skewed,0.0,0.0,219,22.74


Tidak handling outlier karena nilai skewnya kecil

## Feature Selection

### Correlation Test

In [509]:
def check_correlation(X, y, num_cols, cat_cols):
    # Initialize lists to store results
    cols = []
    jenis_col = []
    metode_uji = []
    corr = []
    p_values = []
    interpretasi = []

    # Menguji kolom numerik: Pearson atau Spearman, tergantung pada distribusi
    for col in num_cols:
        # Menguji apakah data normal menggunakan normaltest
        _, p_normal = normaltest(X[col].dropna())

        if p_normal > 0.05:  # Data berdistribusi normal
            corr_coef, p_value = pearsonr(X[col], y)
            metode = 'Pearson'
        else:  # Data tidak berdistribusi normal
            corr_coef, p_value = spearmanr(X[col], y)
            metode = 'Spearman'

        cols.append(col)
        jenis_col.append('numerical')
        metode_uji.append(metode)
        corr.append(corr_coef)
        p_values.append(p_value)
        interpretasi.append('Ada korelasi' if p_value < 0.05 else 'Tidak ada korelasi')

    # Menguji kolom kategorikal menggunakan Kendall
    for col in cat_cols:
        corr_coef, p_value = kendalltau(X[col], y)
        cols.append(col)
        jenis_col.append('categorical')
        metode_uji.append('Kendall Tau')
        corr.append(corr_coef)
        p_values.append(p_value)
        interpretasi.append('Ada korelasi' if p_value < 0.05 else 'Tidak ada korelasi')

    # Membuat DataFrame untuk menampilkan hasil
    result_df = pd.DataFrame({
        'Nama Kolom': cols,
        'Jenis Kolom': jenis_col,
        'Metode Uji': metode_uji,
        'Nilai Korelasi': corr,
        'p_value': p_values,
        'Hasil': interpretasi
    })

    return result_df

In [510]:
check_correlation(X_train_imputed, y_train, num_cols, cat_cols)

,Nama Kolom,Jenis Kolom,Metode Uji,Nilai Korelasi,p_value,Hasil
0,land_size_m2,numerical,Spearman,0.728375,5.688673e-160,Ada korelasi
1,building_size_m2,numerical,Spearman,0.816033,5.183209e-231,Ada korelasi
2,road_width,numerical,Spearman,0.283361,3.066286e-19,Ada korelasi
3,maid_bedroom,numerical,Spearman,0.562537,1.850428e-81,Ada korelasi
4,maid_bathroom,numerical,Spearman,0.510996,3.559139e-65,Ada korelasi
5,kitchen,numerical,Spearman,0.258708,3.430881e-16,Ada korelasi
6,floor_level,numerical,Spearman,0.367561,3.582619e-32,Ada korelasi
7,bedroom,numerical,Spearman,0.568980,1.058615e-83,Ada korelasi
8,bathroom,numerical,Spearman,0.717751,3.029629e-153,Ada korelasi
9,garage,numerical,Spearman,0.481933,3.728907e-57,Ada korelasi


In [511]:
X_train_imputed.columns

Index(['land_size_m2', 'building_size_m2', 'road_width', 'city',
       'property_type', 'certificate', 'furniture', 'house_facing',
       'water_source', 'property_condition', 'bedroom', 'bathroom', 'garage',
       'carport', 'voltage_watt', 'maid_bedroom', 'maid_bathroom', 'kitchen',
       'floor_level'],
      dtype='object')

## Scaling and Encoding

In [512]:
# Making column transformer for preprocessing
transformer = ColumnTransformer([
('scaler', MinMaxScaler(), num_cols),
('encoder_ohe', OneHotEncoder(), cat_cols)
])
transformer

ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                 ['land_size_m2', 'building_size_m2',
                                  'road_width', 'maid_bedroom', 'maid_bathroom',
                                  'kitchen', 'floor_level', 'bedroom',
                                  'bathroom', 'garage', 'carport',
                                  'voltage_watt']),
                                ('encoder_ohe', OneHotEncoder(),
                                 ['city', 'property_type', 'certificate',
                                  'furniture', 'house_facing', 'water_source',
                                  'property_condition'])])

# VI. MODEL DEFINITION

In [513]:
# Making pipeline for LogReg
pipe_KNN = Pipeline([
('transformer', transformer),
('model', KNeighborsRegressor(n_neighbors=5)
 )
])

# Making pipeline for SVR
pipe_SVR = Pipeline([
('transformer', transformer),
('model', SVR(kernel='linear')
)
])

# Making pipeline for Decision Tree
pipe_dt = Pipeline([
('transformer', transformer),
('model', DecisionTreeRegressor(random_state=999))
])

# Making pipeline for Random Forest
pipe_rf = Pipeline([
('transformer', transformer),
('model', RandomForestRegressor(random_state=999))
])

# Making pipeline for ADABoost
pipe_ada = Pipeline([
('transformer', transformer),
('model', AdaBoostRegressor(random_state=999))
])

In [514]:
# Training KNN
pipe_KNN.fit(X_train_imputed, y_train)

# Training SVM
pipe_SVR.fit(X_train_imputed, y_train)

# Training decision tree
pipe_dt.fit(X_train_imputed, y_train)

# Training Random Forest
pipe_rf.fit(X_train_imputed, y_train)

# Training Ada boost
pipe_ada.fit(X_train_imputed, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  ['land_size_m2',
                                                   'building_size_m2',
                                                   'road_width', 'maid_bedroom',
                                                   'maid_bathroom', 'kitchen',
                                                   'floor_level', 'bedroom',
                                                   'bathroom', 'garage',
                                                   'carport', 'voltage_watt']),
                                                 ('encoder_ohe',
                                                  OneHotEncoder(),
                                                  ['city', 'property_type',
                                                   'certificate', 'furniture',
                                                   'house_facing',
                                                   'water_source',
                                                   'property_condition'])])),
                ('model', AdaBoostRegressor(random_state=999))])

In [515]:
def performance_report(all_reports, crossvals, y_train, y_pred_train, y_test, y_pred_test, name):
    '''
    Fungsi ini digunakan untuk melihat RMSE dari setiap model yang digunakan.

    Argumen:
    - all_reports = dictionary yang memuat rmse
    - crossvals = hasil crosval
    - y_train = train target
    - y_pred_train = prediksi train
    - y_test = test target
    - y_pred_test = prediksi test
    - name = nama kolom
    '''
    # list evaluasi train, cv dan test
    score_reports = {
      'train' : mean_absolute_error(y_train, y_pred_train),
      'mean - cv' : crossvals.mean(),
      'std - cv': crossvals.std(),
      'test' : mean_absolute_error(y_test, y_pred_test),
    }
    
    # create new key
    all_reports[name] = score_reports
    
    # mengembalikan all_report
    return all_reports

## Cross-Validation

In [523]:
# crossval knn
knn_train_cross_val = cross_val_score(pipe_KNN,
                                     X_train_imputed,
                                     y_train,
                                     cv=5,
                                     scoring="neg_mean_absolute_error"
                                     )

# Konversi nilai negatif menjadi positif (karena neg_mean_absolute_error memberikan nilai negatif)
rmse_knn_cv = np.abs(knn_train_cross_val)

# predict knn
y_pred_train_knn = pipe_KNN.predict(X_train_imputed)
y_pred_test_knn = pipe_KNN.predict(X_test_imputed)

# crossval svm
svm_train_cross_val = cross_val_score(pipe_SVR,
                                     X_train_imputed,
                                     y_train,
                                     cv=5,
                                     scoring="neg_mean_absolute_error"
                                     )

# Konversi nilai negatif menjadi positif (karena neg_mean_absolute_error memberikan nilai negatif)
rmse_svm_cv = np.abs(svm_train_cross_val)

# predict svm
y_pred_train_svm = pipe_SVR.predict(X_train_imputed)
y_pred_test_svm = pipe_SVR.predict(X_test_imputed)

# crossval dt
dt_train_cross_val = cross_val_score(pipe_dt,
                                     X_train_imputed,
                                     y_train,
                                     cv=5,
                                     scoring="neg_mean_absolute_error"
                                     )

# Konversi nilai negatif menjadi positif (karena neg_mean_absolute_error memberikan nilai negatif)
rmse_dt_cv = np.abs(dt_train_cross_val)

# predict dt
y_pred_train_dt = pipe_dt.predict(X_train_imputed)
y_pred_test_dt = pipe_dt.predict(X_test_imputed)

# crossval rf
rf_train_cross_val = cross_val_score(pipe_rf,
                                     X_train_imputed,
                                     y_train,
                                     cv=5,
                                     scoring="neg_mean_absolute_error"
                                     )

# Konversi nilai negatif menjadi positif (karena neg_mean_absolute_error memberikan nilai negatif)
rmse_rf_cv = np.abs(rf_train_cross_val)

# predict rf
y_pred_train_rf = pipe_rf.predict(X_train_imputed)
y_pred_test_rf = pipe_rf.predict(X_test_imputed)

# crossval ada boost
ada_train_cross_val = cross_val_score(pipe_ada,
                                     X_train_imputed,
                                     y_train,
                                     cv=5,
                                     scoring="neg_mean_absolute_error"
                                     )

# Konversi nilai negatif menjadi positif (karena neg_mean_absolute_error memberikan nilai negatif)
rmse_ada_cv = np.abs(ada_train_cross_val)

# predict ada boost
y_pred_train_ada = pipe_ada.predict(X_train_imputed)
y_pred_test_ada = pipe_ada.predict(X_test_imputed)

# define untuk menyimpan report
all_reports = {}

# knn report
all_reports = performance_report(all_reports,rmse_knn_cv, y_train, y_pred_train_knn, y_test, y_pred_test_knn, 'KNN Baseline (Default Hyperparameter)')
# svm report
all_reports = performance_report(all_reports,rmse_svm_cv, y_train, y_pred_train_svm, y_test, y_pred_test_svm, 'SVR Baseline (Default Hyperparameter)')
# dt report
all_reports = performance_report(all_reports,rmse_dt_cv, y_train, y_pred_train_dt, y_test, y_pred_test_dt, 'DT Baseline (Default Hyperparameter)')
# rf report
all_reports = performance_report(all_reports,rmse_rf_cv, y_train, y_pred_train_rf, y_test, y_pred_test_rf, 'RF Baseline (Default Hyperparameter)')
# rf report
all_reports = performance_report(all_reports,rmse_ada_cv, y_train, y_pred_train_ada, y_test, y_pred_test_ada, 'ADA Baseline (Default Hyperparameter)')

# to df
report = pd.DataFrame(all_reports)

# show
report

,KNN Baseline (Default Hyperparameter),SVR Baseline (Default Hyperparameter),DT Baseline (Default Hyperparameter),RF Baseline (Default Hyperparameter),ADA Baseline (Default Hyperparameter)
train,2471.173091,3973.115439,2.276428,772.819832,7808.833045
mean - cv,3215.613429,4015.582404,2761.034982,2200.503336,6094.341347
std - cv,604.451792,793.586172,609.291769,315.968310,1009.735146
test,3880.787415,4043.092986,3203.966355,1570.054045,8369.905145


## Hyperparameter Tuning

In [517]:
# show pipeline's parameter
pipe_rf.get_params()

{'memory': None,
 'steps': [('transformer',
   ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                    ['land_size_m2', 'building_size_m2',
                                     'road_width', 'maid_bedroom', 'maid_bathroom',
                                     'kitchen', 'floor_level', 'bedroom',
                                     'bathroom', 'garage', 'carport',
                                     'voltage_watt']),
                                   ('encoder_ohe', OneHotEncoder(),
                                    ['city', 'property_type', 'certificate',
                                     'furniture', 'house_facing', 'water_source',
                                     'property_condition'])])),
  ('model', RandomForestRegressor(random_state=999))],
 'verbose': False,
 'transformer': ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                  ['land_size_m2', 'building_size_m2',
                           

In [518]:
# Parameter Grid
param_distributions = {
    'model__n_estimators': [5, 10, 50, 100, 300, 500],
    'model__max_depth': [10, 20, 30, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipe_rf,
    param_distributions=param_distributions,
    n_iter=50,  # Reduce iterations for faster experimentation; adjust for final runs
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    random_state=0  # Ensures reproducibility
)

# VII. MODEL TRAINING

In [519]:
# Fit RandomizedSearchCV
random_search.fit(X_train_imputed, y_train)

# Display Best Results
print("Best Parameters:", random_search.best_params_)
print("Best Score (Negative MAE):", random_search.best_score_)

# Use the Best Model
best_model = random_search.best_estimator_

Best Parameters: {'model__n_estimators': 300, 'model__min_samples_split': 2, 'model__min_samples_leaf': 2, 'model__max_depth': 20}
Best Score (Negative MAE): -2194.438532201234


# VIII. MODEL EVALUATION

In [520]:
# Predict train-set & test-set
y_train_predict = best_model.predict(X_train_imputed)
y_test_predict = best_model.predict(X_test_imputed)

In [521]:
# RMSE (Root Mean Squared Error) Train and Test:
mae_train = mean_absolute_error(y_train, y_train_predict)
mae_test = mean_absolute_error(y_test, y_test_predict)
r2_train = r2_score(y_train, y_train_predict)
r2_test = r2_score(y_test, y_test_predict)

print(f"Mean Absolute Error (MAE) - TRAIN: {mae_train:.2f}")
print(f"Mean Absolute Error (MAE) - TEST: {mae_test:.2f}")
print(f"R-squared (R2 Score): {r2_train:.2f}")
print(f"R-squared (R2 Score): {r2_test:.2f}")

Mean Absolute Error (MAE) - TRAIN: 964.80
Mean Absolute Error (MAE) - TEST: 1638.03
R-squared (R2 Score): 0.93
R-squared (R2 Score): 0.85


# IX. MODEL SAVING

In [522]:
# Menyimpan model ke file
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Model berhasil disimpan!")

Model berhasil disimpan!
